In [13]:
from pathlib import Path
import pandas as pd
import duckdb

data_root = Path("D:\\데이터\\건축데이터 건축허브 개방데이터")
path_DB = data_root / "건축물대장_2025년_02월.db"

results_dir = Path("../results")

In [14]:
# Open a DuckDB connection
con = duckdb.connect(database=path_DB, read_only=True)

# print the list of tables in the database
tables = con.execute("SHOW TABLES").fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])
    print()

# show heads of the tables
for table in tables:
    table_name = table[0]
    print(f"Head of {table_name}:")
    # print(con.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchdf())
    print(con.sql(f"SELECT * FROM {table_name} LIMIT 5"))
    print()

Tables in the database:
기본개요

층별개요

표제부

Head of 기본개요:
┌────────────────────┬─────────────────────────┬────────────────┬───────────────────┬────────────────┬───────────────────┬───────────────────────────────────┬─────────────────────────────────────────┬─────────┬─────────────┬─────────────┬────────────────┬─────────┬─────────┬───────────┬─────────┬─────────┬───────────┬──────────────────┬────────────────────┬──────────────────────┬──────────────┬──────────────┬───────────┬───────────┬───────────┬──────────────┬──────────────┬──────────────┬───────────┐
│ 관리_건축물대장_PK │ 관리_상위_건축물대장_PK │ 대장_구분_코드 │ 대장_구분_코드_명 │ 대장_종류_코드 │ 대장_종류_코드_명 │             대지_위치             │            도로명_대지_위치             │ 건물_명 │ 시군구_코드 │ 법정동_코드 │ 대지_구분_코드 │   번    │   지    │ 특수지_명 │  블록   │  로트   │ 외필지_수 │ 새주소_도로_코드 │ 새주소_법정동_코드 │ 새주소_지상지하_코드 │ 새주소_본_번 │ 새주소_부_번 │ 지역_코드 │ 지구_코드 │ 구역_코드 │ 지역_코드_명 │ 지구_코드_명 │ 구역_코드_명 │ 생성_일자 │
│      varchar       │         varchar         │    varchar     │      varchar      

In [15]:
# Create a list to store the table names and their record counts
table_counts = []

# Iterate through the tables and count the records
for table in tables:
    table_name = table[0]
    count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    table_counts.append({"Table Name": table_name, "Record Count": count})

# Convert the list to a DataFrame
record_counts_df = pd.DataFrame(table_counts)

# Display the DataFrame
display(record_counts_df)

,Table Name,Record Count
0,기본개요,27994157
1,층별개요,21048715
2,표제부,8027067


In [16]:
# Create a temporary view with the selected columns
층별개요_필터링 = con.sql("""
    SELECT 
        "주_용도_코드",
        "주_용도_코드_명",
        "기타_용도"
    FROM 층별개요
    WHERE "기타_용도" IS NOT NULL AND "기타_용도" != ''
        AND "주_용도_코드_명" IS NOT NULL AND "주_용도_코드_명" != ''
""")
# Count the records in the filtered view
con.sql("SELECT COUNT(*) FROM 층별개요_필터링").fetchone()[0]

20985868

In [17]:
# Perform value count using SQL
value_counts_sql = con.sql("""
    SELECT 
        "주_용도_코드",
        "주_용도_코드_명",
        "기타_용도",
        COUNT(*) AS "Count"
    FROM 층별개요_필터링
    GROUP BY "주_용도_코드", "주_용도_코드_명", "기타_용도"
    ORDER BY "Count" DESC
""")

# Fetch and display the result
value_counts_df = value_counts_sql.fetchdf()
display(value_counts_df.head(5))
# Save the top 30 rows to a CSV file
# value_counts_df.head(30).to_csv(
#     results_dir / "표제부_용도_기재내용_top30.csv", index=False, encoding="utf-8-sig"
# )

,주_용도_코드,주_용도_코드_명,기타_용도,Count
0,01001,단독주택,주택,3192276
1,02001,아파트,아파트,1749831
2,01001,단독주택,단독주택,1568151
3,02001,아파트,공동주택(아파트),703628
4,18001,창고,창고,462082


In [18]:
# Create a temporary view to split the '기타_용도' column into arrays using a regular expression
value_counts_split = con.sql("""
    SELECT 
        *,
        regexp_split_to_array(lower("기타_용도"), '[^\\p{L}\\d]+') AS "기타_용도_분리"
    FROM value_counts_sql
""")

# Fetch and display the result
value_counts_split.fetchdf()

,주_용도_코드,주_용도_코드_명,기타_용도,Count,기타_용도_분리
0,01001,단독주택,주택,3192276,[주택]
1,02001,아파트,아파트,1749831,[아파트]
2,01001,단독주택,단독주택,1568151,[단독주택]
3,02001,아파트,공동주택(아파트),703628,"[공동주택, 아파트, ]"
4,18001,창고,창고,462082,[창고]
...,...,...,...,...,...
771959,10103,고등학교,"식당,세미나실,체력단련실",1,"[식당, 세미나실, 체력단련실]"
771960,17999,기타공장,공장(부대창고)/주차장/기계실/전기실),1,"[공장, 부대창고, 주차장, 기계실, 전기실, ]"
771961,11201,노인복지시설,노유자시설(노인복지시설(유료노인요양시설),1,"[노유자시설, 노인복지시설, 유료노인요양시설, ]"
771962,03022,마을공동작업소,실용실습장,1,[실용실습장]


In [19]:
# Create a temporary view to split the '기타_용도' column into rows using a regular expression
value_counts_split_table = con.sql("""
    SELECT 
        *,
        regexp_split_to_table(lower("기타_용도"), '[^\\p{L}\\d]+') AS "기타_용도_분리"
    FROM value_counts_sql
""")

# Fetch and display the result
value_counts_split_table.fetchdf()

,주_용도_코드,주_용도_코드_명,기타_용도,Count,기타_용도_분리
0,01001,단독주택,주택,3192276,주택
1,02001,아파트,아파트,1749831,아파트
2,01001,단독주택,단독주택,1568151,단독주택
3,02001,아파트,공동주택(아파트),703628,공동주택
4,02001,아파트,공동주택(아파트),703628,아파트
...,...,...,...,...,...
2467662,10104,대학교,"강의실,실습실,사무실",1,사무실
2467663,10104,대학교,"준비실,어학실,시청각실,강의실",1,준비실
2467664,10104,대학교,"준비실,어학실,시청각실,강의실",1,어학실
2467665,10104,대학교,"준비실,어학실,시청각실,강의실",1,시청각실


In [20]:
# Group by valid (5 digit) '주_용도_코드',
# sum the 'Count' column, and filter where the sum is >= 100
class_counts_sql = con.sql("""
    SELECT
        "주_용도_코드",
        "주_용도_코드_명",
        SUM("Count") AS "Total_Count"
    FROM value_counts_sql
    WHERE LENGTH("주_용도_코드") = 5 AND "주_용도_코드" ~ '^\\d+$'
    GROUP BY "주_용도_코드", "주_용도_코드_명"
    HAVING SUM("Count") >= 200
    ORDER BY "Total_Count" DESC
""")

# Fetch and display the result
class_counts_df = class_counts_sql.fetchdf().astype({"Total_Count": int})
display(class_counts_df)
# Save to a CSV file
class_counts_df.to_csv(
    results_dir / "층_naive_bayes_클래스별_문서빈도.csv",
    index=False,
    encoding="utf-8-sig",
)

,주_용도_코드,주_용도_코드_명,Total_Count
0,01001,단독주택,6107851
1,02001,아파트,2866578
2,01003,다가구주택,1922416
3,02003,다세대주택,1260327
4,03001,소매점,902487
...,...,...,...
402,14203,신문사,216
403,04404,결혼상담소,213
404,21201,가축용운동시설,211
405,19014,유독물저장소,207


In [21]:
# Group by '기타_용도_분리',
# sum the 'Count' column, and filter where the sum is >= 100
word_counts_sql = con.sql("""
    SELECT
        "기타_용도_분리",
        SUM("Count") AS "Total_Count"
    FROM value_counts_split_table
    WHERE "기타_용도_분리" IS NOT NULL AND "기타_용도_분리" != ''
    GROUP BY "기타_용도_분리"
    HAVING SUM("Count") >= 200
    ORDER BY "Total_Count" DESC
""")

# Fetch and display the result
word_counts_df = word_counts_sql.fetchdf().astype({"Total_Count": int})
display(word_counts_df)
# Save to a CSV file
word_counts_df.to_csv(
    results_dir / "층_naive_bayes_단어별_문서빈도.csv",
    index=False,
    encoding="utf-8-sig",
)

,기타_용도_분리,Total_Count
0,주택,3857116
1,아파트,2583267
2,단독주택,2216874
3,창고,1067192
4,공동주택,1004938
...,...,...
2003,생활권수련시설,201
2004,1996,200
2005,102,200
2006,가정보육시설,200


In [22]:
# Construct training data with '주_용도_코드' and '기타_용도_분리'
# Filter with class_counts_sql and word_counts_sql
train_data_sql = con.sql("""
    SELECT
        t."주_용도_코드" AS class,
        t."기타_용도_분리" AS word,
        t."Count" AS weight,
    FROM value_counts_split_table t
    JOIN class_counts_sql c
        ON t."주_용도_코드" = c."주_용도_코드"
    JOIN word_counts_sql w
        ON t."기타_용도_분리" = w."기타_용도_분리"
    WHERE t.Count >= 200
    ORDER BY t."Count" DESC
""")
train_df = train_data_sql.fetchdf()
train_df

,class,word,weight
0,01001,주택,3192276
1,02001,아파트,1749831
2,01001,단독주택,1568151
3,02001,공동주택,703628
4,02001,아파트,703628
...,...,...,...
5864,03199,공공업무시설,200
5865,01003,물탱크실,200
5866,01003,연면적제외,200
5867,04025,인터넷컴퓨터게임시설제공업,200


In [23]:
import duckdb
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB


# 2. Vectorize & train

# The multinomial Naive Bayes classifier is suitable for classification with
# discrete features (e.g., word counts for text classification). The multinomial
# distribution normally requires integer feature counts. However, in practice,
# fractional counts such as tf-idf may also work.

# word is already a word, but no harm from count vectorization
# regex does not support unicode, so we need to customize the pattern
vectorizer = CountVectorizer(token_pattern=r"[\\w\\d가-힣]+")
X = vectorizer.fit_transform(train_df["word"])
y = train_df["class"]
weights = train_df["weight"]
clf = MultinomialNB(alpha=1.0)
clf.fit(X, y, sample_weight=weights)

MultinomialNB()

In [24]:
vectorizer.vocabulary_

{'주택': 632,
 '아파트': 432,
 '단독주택': 212,
 '공동주택': 51,
 '창고': 669,
 '다세대주택': 206,
 '소매점': 395,
 '일반음식점': 541,
 '축사': 693,
 '다가구주택': 198,
 '제': 594,
 '종근린생활시설': 612,
 '사무소': 363,
 '세대': 390,
 '공장': 53,
 '점포': 584,
 '근린생활시설': 100,
 '가구': 2,
 '변소': 312,
 '농가주택': 159,
 '사무실': 364,
 '제조업소': 603,
 '부속사': 346,
 '연립주택': 458,
 '업무시설': 450,
 '오피스텔': 480,
 '계단실': 33,
 '화장실': 771,
 '물치': 285,
 '여관': 453,
 '학원': 754,
 '휴게음식점': 779,
 '의원': 526,
 '퇴비사': 723,
 '주차장': 629,
 '연면적제외': 462,
 '일반공장': 535,
 '우사': 496,
 '대피소': 226,
 '부속': 344,
 '다중주택': 210,
 '일반창고': 543,
 '마을회관': 265,
 '대중음식점': 225,
 '창고시설': 676,
 '교회': 86,
 '지하실': 654,
 '보일러실': 329,
 '다가구용단독주택': 196,
 '지하주차장': 655,
 '돈사': 249,
 '기타공장': 120,
 '경비실': 30,
 '수리점': 403,
 '다방': 204,
 '숙박시설': 406,
 '다가구': 193,
 '농업생산시설': 184,
 '부속창고': 350,
 '노인복지시설': 147,
 '국방': 92,
 '군사시설': 94,
 '노유자시설': 145,
 '미용원': 295,
 '교육연구시설': 83,
 '고시원': 41,
 '계사': 37,
 '종교시설': 608,
 '부속건물': 345,
 '동': 250,
 '식물관련시설': 417,
 '물품': 289,
 '제조공장': 600,
 '지하주차장연결통로': 656,
 '생활숙박시설

In [25]:
# test the classifier
test_text = "물탱크,EV기계실(연면적 제외)"
X_test = vectorizer.transform([test_text])
X_test.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 

In [26]:
clf.predict(X_test)[0]

np.str_('01003')

In [27]:
# 3. Register a prediction UDF
def nb_predict(text: str) -> str:
    x = vectorizer.transform([text])
    return clf.predict(x)[0]


# Remove the function if it already exists
try:
    con.remove_function("nb_predict")
except Exception as e:
    if "No function by the name of" not in str(e):
        raise

# Register the function
con.create_function("nb_predict", nb_predict)

In [28]:
preds = con.sql("""
  SELECT
    *,
    nb_predict("기타_용도") AS predicted_label,
    CASE 
      WHEN "주_용도_코드" = predicted_label THEN 1 
      ELSE 0 
    END AS is_correct
  FROM value_counts_sql
""")
preds_label = con.sql("""
  SELECT
    preds.*,
    class_counts_sql."주_용도_코드_명" AS "predicted_label_name",
  FROM preds
  LEFT JOIN class_counts_sql
    ON preds."predicted_label" = class_counts_sql."주_용도_코드"
""")
preds_df = preds_label.fetchdf()
preds_df

,주_용도_코드,주_용도_코드_명,기타_용도,Count,predicted_label,is_correct,predicted_label_name
0,17100,일반공장,비가림처마,1,01001,0,단독주택
1,03109,공공도서관,정화조점검층,1,01001,0,단독주택
2,17100,일반공장,"일반공장 (창고,물탱크실)(말소)",1,17100,1,일반공장
3,04005,제조업소,교반실,1,01001,0,단독주택
4,02003,다세대주택,빗물탱크(연면적제외),1,02003,1,다세대주택
...,...,...,...,...,...,...,...
771959,03199,기타공공시설,제1종근린생활시설(숙소),1,04040,0,교습소
771960,19002,액화석유가스충전소,"액화석유가스충전소(관리실,숙소)",1,19002,1,액화석유가스충전소
771961,14299,기타일반업무시설,"기타일반업무시설,제1종근린생활시설",1,04040,0,교습소
771962,03005,의원,제1종근린생활시설(의원)/제2종근린생활시설(교습소),1,04040,0,교습소


In [29]:
# Save preds to a CSV file
preds_df.to_csv(
    results_dir / "층_naive_bayes_예측결과.csv",
    index=False,
    encoding="utf-8-sig",
)